In [61]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn import datasets
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

ModuleNotFoundError: No module named 'ml_metrics'

# Case 5: Imputation

Andrew Larsen  
Matthew Chinchilla  
Rikel Djoko  

## Baseline Model Development

In [12]:
# Baseline Lin Reg Model Here
california = datasets.fetch_california_housing()
cal = pd.DataFrame(california.data)
cal.columns = california.feature_names
cal['MedInc'] = california.target

In [13]:
cal.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,4.526,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,3.585,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,3.521,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,3.413,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.422,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [32]:
#check for NA
cal.isna().sum()

MedInc        0
HouseAge      0
AveRooms      0
AveBedrms     0
Population    0
AveOccup      0
Latitude      0
Longitude     0
dtype: int64

In [35]:
#define variable and  split data
cols = ['HouseAge','AveRooms','AveBedrms','Population','AveOccup','Latitude','Longitude']
X = cal[cols]
y = cal["MedInc"]
#split the data
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.4, random_state = 0)

In [55]:
#train model using linear regression
lin_model = LinearRegression()
lin_model.fit(train_X, train_y)
coef_df = pd.DataFrame(lin_model.coef_, cols, columns = ["coef"])
coef_df

,coef
HouseAge,0.005633
AveRooms,0.363187
AveBedrms,-1.348481
Population,-0.000011
AveOccup,-0.003401
Latitude,-0.742928
Longitude,-0.738761


In [43]:
##predict
y_predict = lin_model.predict(test_X)

In [63]:
##perfromance evaluation using   R  square, MSE, MAE, RMSE
# r_square = r2_score(test_y, y_predict)

orig_mae = mean_absolute_error(test_y, y_predict)
orig_mse = mean_squared_error(test_y, y_predict)
# orig_rmse_val = rmse(test_y, y_predict)
orig_r2 = r2_score(test_y, y_predict)
print("MAE: %.3f"%orig_mae)
print("MSE:  %.3f"%orig_mse)
# print("RMSE:  %.3f"%orig_rmse_val)
print("R2:  %.3f"%orig_r2)

MAE: 0.684
MSE:  0.825
R2:  0.382


In [65]:
perf_frame = pd.DataFrame({'data':'original',
                   'imputation':'none',
                   'mae': orig_mae, 
                   'mse': orig_mse, 
                   'R2':orig_r2,
                   'mae_diff':np.nan,
                   'mse_diff':np.nan,
                   'R2_diff':np.nan}, index=[0])
perf_frame

,data,imputation,mae,mse,R2,mae_diff,mse_diff,R2_diff
0,original,none,0.684403,0.824646,0.381605,NaN,NaN,NaN


In [95]:
type(cal)

pandas.core.frame.DataFrame

In [101]:
def LR_pipeline(df):
    
    #1 set the test and trainning set X and Y
    train_X = df.iloc[train_X_index, 1:]
    train_y = df["MedInc"].iloc[train_y_index]
    test_X = df.iloc[test_X_index, 1:].dropna()
    test_y = df["MedInc"].iloc[test_y_index]
    #2  create and fit the model using LR
    lin_model = LinearRegression()
    lin_model.fit(train_X, train_y)
    coef_df = pd.DataFrame(lin_model.coef_, cols, columns = ["coef"])
    print(coef_df)
    #3  predict using 
    y_predict = lin_model.predict(test_X)
    # 4 evaluation and performance
    orig_mae = mean_absolute_error(test_y, y_predict)
    orig_mse = mean_squared_error(test_y, y_predict)
    perf_frame = pd.DataFrame({'data':'original',
                   'imputation':'none',
                   'mae': orig_mae, 
                   'mse': orig_mse, 
                   'R2':orig_r2,
                   'mae_diff':np.nan,
                   'mse_diff':np.nan,
                   'R2_diff':np.nan}, index=[0])
    return perf_frame
    

In [103]:
# Using  PIPELINE TO RUN PREDICTION
LR_pipeline(cal)

                coef
HouseAge    0.005555
AveRooms    0.361375
AveBedrms  -1.340644
Population -0.000011
AveOccup   -0.003397
Latitude   -0.741965
Longitude  -0.738027


,data,imputation,mae,mse,R2,mae_diff,mse_diff,R2_diff
0,original,none,0.686196,0.827301,0.379614,NaN,NaN,NaN


## Missing Completely at Random

### mcar .01

In [84]:
california = datasets.fetch_california_housing()
cal = pd.DataFrame(california.data)
cal.columns = california.feature_names
cal['MedInc'] = california.target
cal.head()
mcardf = cal
nidx = round(.01 * len(cal))
dropidxs = np.random.choice(len(cal), nidx, replace=False)
mcardf.loc[dropidxs, 'AveRooms'] = np.nan
print(mcardf.isna().sum() / len(cal))

MedInc        0.000000
HouseAge      0.000000
AveRooms      0.009981
AveBedrms     0.000000
Population    0.000000
AveOccup      0.000000
Latitude      0.000000
Longitude     0.000000
dtype: float64


In [85]:
mcardf['AveRooms'] = mcardf['AveRooms'].fillna(mcardf['AveRooms'].mean())
print(mcardf.isna().sum() / len(cal))

MedInc        0.0
HouseAge      0.0
AveRooms      0.0
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [105]:
# Create Lin Reg Model Here
LR_pipeline(mcardf)

                coef
HouseAge    0.005555
AveRooms    0.361375
AveBedrms  -1.340644
Population -0.000011
AveOccup   -0.003397
Latitude   -0.741965
Longitude  -0.738027


,data,imputation,mae,mse,R2,mae_diff,mse_diff,R2_diff
0,original,none,0.686196,0.827301,0.379614,NaN,NaN,NaN


### mcar .05

In [106]:
california = datasets.fetch_california_housing()
cal = pd.DataFrame(california.data)
cal.columns = california.feature_names
cal['MedInc'] = california.target
cal.head()
mcardf = cal
nidx = round(.05 * len(cal))
dropidxs = np.random.choice(len(cal), nidx, replace=False)
mcardf.loc[dropidxs, 'AveRooms'] = np.nan
print(mcardf.isna().sum() / len(cal))

MedInc        0.00
HouseAge      0.00
AveRooms      0.05
AveBedrms     0.00
Population    0.00
AveOccup      0.00
Latitude      0.00
Longitude     0.00
dtype: float64


In [107]:
mcardf['AveRooms'] = mcardf['AveRooms'].fillna(mcardf['AveRooms'].mean())
print(mcardf.isna().sum() / len(cal))

MedInc        0.0
HouseAge      0.0
AveRooms      0.0
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [108]:
# Create Lin Reg Model Here
LR_pipeline(mcardf)

                coef
HouseAge    0.004653
AveRooms    0.343463
AveBedrms  -1.241932
Population -0.000016
AveOccup   -0.003276
Latitude   -0.737082
Longitude  -0.734595


,data,imputation,mae,mse,R2,mae_diff,mse_diff,R2_diff
0,original,none,0.693893,0.856887,0.357428,NaN,NaN,NaN


### mcar .1

In [109]:
california = datasets.fetch_california_housing()
cal = pd.DataFrame(california.data)
cal.columns = california.feature_names
cal['MedInc'] = california.target
cal.head()
mcardf = cal
nidx = round(.1 * len(cal))
dropidxs = np.random.choice(len(cal), nidx, replace=False)
mcardf.loc[dropidxs, 'AveRooms'] = np.nan
print(mcardf.isna().sum() / len(cal))

MedInc        0.0
HouseAge      0.0
AveRooms      0.1
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [110]:
mcardf['AveRooms'] = mcardf['AveRooms'].fillna(mcardf['AveRooms'].mean())
print(mcardf.isna().sum() / len(cal))

MedInc        0.0
HouseAge      0.0
AveRooms      0.0
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [111]:
# Create Lin Reg Model Here
LR_pipeline(mcardf)

                coef
HouseAge    0.003931
AveRooms    0.308138
AveBedrms  -1.041675
Population -0.000019
AveOccup   -0.003191
Latitude   -0.734739
Longitude  -0.735041


,data,imputation,mae,mse,R2,mae_diff,mse_diff,R2_diff
0,original,none,0.700859,0.894185,0.329458,NaN,NaN,NaN


### mcar .2

In [112]:
california = datasets.fetch_california_housing()
cal = pd.DataFrame(california.data)
cal.columns = california.feature_names
cal['MedInc'] = california.target
cal.head()
mcardf = cal
nidx = round(.2 * len(cal))
dropidxs = np.random.choice(len(cal), nidx, replace=False)
mcardf.loc[dropidxs, 'AveRooms'] = np.nan
print(mcardf.isna().sum() / len(cal))

MedInc        0.0
HouseAge      0.0
AveRooms      0.2
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [113]:
mcardf['AveRooms'] = mcardf['AveRooms'].fillna(mcardf['AveRooms'].mean())
print(mcardf.isna().sum() / len(cal))

MedInc        0.0
HouseAge      0.0
AveRooms      0.0
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [114]:
# Create Lin Reg Model Here
LR_pipeline(mcardf)

                coef
HouseAge    0.002263
AveRooms    0.261029
AveBedrms  -0.766870
Population -0.000025
AveOccup   -0.002318
Latitude   -0.728145
Longitude  -0.731194


,data,imputation,mae,mse,R2,mae_diff,mse_diff,R2_diff
0,original,none,0.713694,0.894149,0.329486,NaN,NaN,NaN


### mcar .33

In [115]:
california = datasets.fetch_california_housing()
cal = pd.DataFrame(california.data)
cal.columns = california.feature_names
cal['MedInc'] = california.target
cal.head()
mcardf = cal
nidx = round(.33 * len(cal))
dropidxs = np.random.choice(len(cal), nidx, replace=False)
mcardf.loc[dropidxs, 'AveRooms'] = np.nan
print(mcardf.isna().sum() / len(cal))

MedInc        0.00000
HouseAge      0.00000
AveRooms      0.32999
AveBedrms     0.00000
Population    0.00000
AveOccup      0.00000
Latitude      0.00000
Longitude     0.00000
dtype: float64


In [116]:
mcardf['AveRooms'] = mcardf['AveRooms'].fillna(mcardf['AveRooms'].mean())
print(mcardf.isna().sum() / len(cal))

MedInc        0.0
HouseAge      0.0
AveRooms      0.0
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [118]:
# Create Lin Reg Model Here
LR_pipeline(mcardf)

                coef
HouseAge    0.001478
AveRooms    0.252777
AveBedrms  -0.778968
Population -0.000031
AveOccup   -0.002800
Latitude   -0.716228
Longitude  -0.719967


,data,imputation,mae,mse,R2,mae_diff,mse_diff,R2_diff
0,original,none,0.731284,0.947026,0.289834,NaN,NaN,NaN


### mcar .5

In [119]:
california = datasets.fetch_california_housing()
cal = pd.DataFrame(california.data)
cal.columns = california.feature_names
cal['MedInc'] = california.target
cal.head()
mcardf = cal
nidx = round(.5 * len(cal))
dropidxs = np.random.choice(len(cal), nidx, replace=False)
mcardf.loc[dropidxs, 'AveRooms'] = np.nan
print(mcardf.isna().sum() / len(cal))

MedInc        0.0
HouseAge      0.0
AveRooms      0.5
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [120]:
mcardf['AveRooms'] = mcardf['AveRooms'].fillna(mcardf['AveRooms'].mean())
print(mcardf.isna().sum() / len(cal))

MedInc        0.0
HouseAge      0.0
AveRooms      0.0
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [121]:
# Create Lin Reg Model Here
LR_pipeline(mcardf)

                coef
HouseAge    0.000237
AveRooms    0.179855
AveBedrms  -0.131048
Population -0.000032
AveOccup   -0.002177
Latitude   -0.743708
Longitude  -0.753652


,data,imputation,mae,mse,R2,mae_diff,mse_diff,R2_diff
0,original,none,0.751427,0.9788,0.266007,NaN,NaN,NaN


## Missing at Random

### MAR .1

In [122]:
california = datasets.fetch_california_housing()
cal = pd.DataFrame(california.data)
cal.columns = california.feature_names
cal['MedInc'] = california.target
cal.head()
mardf = cal
idxs = mardf[mardf['AveOccup'] > mardf['AveOccup'].quantile(.5)]
dropidxs = idxs.sample(frac = .2, replace = False)
mardf.loc[dropidxs.index, 'AveRooms'] = np.nan
print(mardf.isna().sum() / len(mardf))

MedInc        0.0
HouseAge      0.0
AveRooms      0.1
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [123]:
mardf['AveRooms'] = mardf['AveRooms'].fillna(mardf['AveRooms'].mean())
print(mardf.isna().sum() / len(cal))

MedInc        0.0
HouseAge      0.0
AveRooms      0.0
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [124]:
# Create Lin Reg Model Here
LR_pipeline(mardf)

                coef
HouseAge    0.004093
AveRooms    0.344051
AveBedrms  -1.262920
Population -0.000018
AveOccup   -0.002998
Latitude   -0.741444
Longitude  -0.741425


,data,imputation,mae,mse,R2,mae_diff,mse_diff,R2_diff
0,original,none,0.701959,0.859592,0.3554,NaN,NaN,NaN


### MAR .2

In [125]:
california = datasets.fetch_california_housing()
cal = pd.DataFrame(california.data)
cal.columns = california.feature_names
cal['MedInc'] = california.target
cal.head()
mardf = cal
idxs = mardf[mardf['AveOccup'] > mardf['AveOccup'].quantile(.5)]
dropidxs = idxs.sample(frac = .4, replace = False)
mardf.loc[dropidxs.index, 'AveRooms'] = np.nan
print(mardf.isna().sum() / len(mardf))

MedInc        0.0
HouseAge      0.0
AveRooms      0.2
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [126]:
mardf['AveRooms'] = mardf['AveRooms'].fillna(mardf['AveRooms'].mean())
print(mardf.isna().sum() / len(cal))

MedInc        0.0
HouseAge      0.0
AveRooms      0.0
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [127]:
# Create Lin Reg Model Here
LR_pipeline(mardf)

                coef
HouseAge    0.001889
AveRooms    0.315235
AveBedrms  -1.127620
Population -0.000027
AveOccup   -0.003236
Latitude   -0.740392
Longitude  -0.743030


,data,imputation,mae,mse,R2,mae_diff,mse_diff,R2_diff
0,original,none,0.719664,0.894152,0.329484,NaN,NaN,NaN


### MAR .3

In [128]:
california = datasets.fetch_california_housing()
cal = pd.DataFrame(california.data)
cal.columns = california.feature_names
cal['MedInc'] = california.target
cal.head()
mardf = cal
idxs = mardf[mardf['AveOccup'] > mardf['AveOccup'].quantile(.5)]
dropidxs = idxs.sample(frac = .6, replace = False)
mardf.loc[dropidxs.index, 'AveRooms'] = np.nan
print(mardf.isna().sum() / len(mardf))

MedInc        0.0
HouseAge      0.0
AveRooms      0.3
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [129]:
mardf['AveRooms'] = mardf['AveRooms'].fillna(mardf['AveRooms'].mean())
print(mardf.isna().sum() / len(cal))

MedInc        0.0
HouseAge      0.0
AveRooms      0.0
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [130]:
# Create Lin Reg Model Here
LR_pipeline(mardf)

                coef
HouseAge    0.000549
AveRooms    0.285330
AveBedrms  -0.986402
Population -0.000033
AveOccup   -0.002148
Latitude   -0.739915
Longitude  -0.746948


,data,imputation,mae,mse,R2,mae_diff,mse_diff,R2_diff
0,original,none,0.736463,0.931859,0.301207,NaN,NaN,NaN


## Missing "Not at" Random

### MNAR .25

In [131]:
california = datasets.fetch_california_housing()
cal = pd.DataFrame(california.data)
cal.columns = california.feature_names
cal['MedInc'] = california.target
cal.head()
mnardf = cal
idxs = (mnardf['AveRooms'] > mnardf['AveRooms'].quantile(.75))
mnardf.loc[idxs, 'AveRooms'] = np.nan
print(mnardf.isna().sum() / len(mnardf))

MedInc        0.000000
HouseAge      0.000000
AveRooms      0.249952
AveBedrms     0.000000
Population    0.000000
AveOccup      0.000000
Latitude      0.000000
Longitude     0.000000
dtype: float64


In [132]:
mnardf['AveRooms'] = mnardf['AveRooms'].fillna(mnardf['AveRooms'].mean())
print(mnardf.isna().sum() / len(cal))

MedInc        0.0
HouseAge      0.0
AveRooms      0.0
AveBedrms     0.0
Population    0.0
AveOccup      0.0
Latitude      0.0
Longitude     0.0
dtype: float64


In [133]:
# Create Lin Reg Model Here
LR_pipeline(mardf)

                coef
HouseAge    0.000549
AveRooms    0.285330
AveBedrms  -0.986402
Population -0.000033
AveOccup   -0.002148
Latitude   -0.739915
Longitude  -0.746948


,data,imputation,mae,mse,R2,mae_diff,mse_diff,R2_diff
0,original,none,0.736463,0.931859,0.301207,NaN,NaN,NaN


## Conclusion